In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from pandas import MultiIndex, Int64Index
# models

import optuna
import pandas as pd
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag
from sklearn.model_selection import train_test_split
import bokeh



/var/folders/32/zv3nm8_12xb9ytd7vk7nnr6w0000gn/T/ipykernel_90799/2656712151.py:8: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#!pip3 install xgboost
#!pip3 install optuna
#!pip install autogluon --upgrade
# !pip install optuna
# !pip install scipy
# !pip install autogluon
# !pip install xgboost
# !python3.9 -m pip install --upgrade pip
#!pip install bokeh==2.0.1
#!pip install jupyter --upgrade


In [2]:
df_train = pd.read_csv('../data/train_encoded_1.csv')
df_test = pd.read_csv('../data/test_encoded_1.csv')

In [3]:
np.random.seed(0)
train, valid = train_test_split(df_train, test_size=0.2)

In [4]:
combined = df_train.drop(['price', 'size_sqft'], axis=1)
X_train = train.drop(['price', 'size_sqft'], axis=1)

X_valid = valid.drop(['price', 'size_sqft'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()
df_test= df_test.drop(['size_sqft'], axis=1)

metric_reg = 'mean_absolute_error'
label_reg = 'price_per_sqft'
train_data_reg = TabularDataset(X_train) 
valid_data_reg = TabularDataset(X_valid)
test_data_reg = TabularDataset(df_test)
combined = TabularDataset(combined)



Tabular Predictor

In [6]:
predictor = TabularPredictor(label = label_reg, eval_metric = metric_reg, path = 'Autogluon_plain')
predictor_reg = predictor.fit(combined)


Beginning AutoGluon training ...
AutoGluon will save models to "Autogluon_plain/"
AutoGluon Version:  0.5.2
Python Version:     3.9.7
Operating System:   Darwin
Train Data Rows:    20147
Train Data Columns: 31
Label Column: price_per_sqft
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (4486.890450010187, 447.8293983244478, 1719.76428, 982.28364)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5774.22 MB
	Train Data (Original)  Memory Usage: 5.0 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata

[1000]	valid_set's l1: 142.873
[2000]	valid_set's l1: 140.912
[3000]	valid_set's l1: 140.689


	-140.5774	 = Validation score   (-mean_absolute_error)
	3.69s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 140.175
[2000]	valid_set's l1: 138.914


	-138.8473	 = Validation score   (-mean_absolute_error)
	2.43s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-134.0851	 = Validation score   (-mean_absolute_error)
	6.7s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	-134.8971	 = Validation score   (-mean_absolute_error)
	162.45s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-132.1399	 = Validation score   (-mean_absolute_error)
	3.26s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-149.7377	 = Validation score   (-mean_absolute_error)
	21.2s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost ...
/Users/nishita/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/nishita/min

In [7]:
#/Users/nishita/Documents/Semester 1/CS5228/project/notebooks/AutogluonModels/ag-20221027_054028
folderName = "Autogluon_plain/"
predictor = TabularPredictor.load(folderName)
predictor.fit_hyperparameters_

{'NN_TORCH': {},
 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
  {},
  'GBMLarge'],
 'CAT': {},
 'XGB': {},
 'FASTAI': {},
 'RF': [{'criterion': 'gini',
   'ag_args': {'name_suffix': 'Gini',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'entropy',
   'ag_args': {'name_suffix': 'Entr',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'squared_error',
   'ag_args': {'name_suffix': 'MSE',
    'problem_types': ['regression', 'quantile']}}],
 'XT': [{'criterion': 'gini',
   'ag_args': {'name_suffix': 'Gini',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'entropy',
   'ag_args': {'name_suffix': 'Entr',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'squared_error',
   'ag_args': {'name_suffix': 'MSE',
    'problem_types': ['regression', 'quantile']}}],
 'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}},
  {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}]}

In [8]:
predictor.evaluate(valid_data_reg)

Evaluation: mean_absolute_error on test data: -109.27624442550133
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -109.27624442550133,
    "root_mean_squared_error": -220.83813362474177,
    "mean_squared_error": -48769.481262859365,
    "r2": 0.9505668121922949,
    "pearsonr": 0.9749931626958129,
    "median_absolute_error": -53.19999999999999
}


{'mean_absolute_error': -109.27624442550133,
 'root_mean_squared_error': -220.83813362474177,
 'mean_squared_error': -48769.481262859365,
 'r2': 0.9505668121922949,
 'pearsonr': 0.9749931626958129,
 'median_absolute_error': -53.19999999999999}

In [9]:
#With target encoding and less one hot encoding.
predictor = TabularPredictor.load(folderName)
results = predictor.fit_summary(show_plot=True)

def writePredictionsToFile(modelName, filename, folder = folderName, refit = False):
    if(refit == True):
        modelName = modelName +  '_FULL'    
    y_pred = predictor.predict(test_data_reg, model=modelName)
    yTest = y_pred * df_test_size_sqft
    yTest = pd.DataFrame(yTest, columns=['Predicted'], index=df_test.index)
    yTest.to_csv(f'{folder}/predictions/{filename}.csv')


*** Summary of fit() ***
Estimated performance of each model:
                  model   score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2 -128.219325       1.420413  232.412809                0.002624           0.360921            2       True         12
1         ExtraTreesMSE -132.139915       0.076599    3.255411                0.076599           3.255411            1       True          7
2         LightGBMLarge -133.672026       0.039147    2.980504                0.039147           2.980504            1       True         11
3       RandomForestMSE -134.085114       0.078687    6.702206                0.078687           6.702206            1       True          5
4              CatBoost -134.897091       0.010550  162.453468                0.010550         162.453468            1       True          6
5               XGBoost -137.024765       0.047590    3.279760                0.047590      

In [10]:
predictor.leaderboard(valid_data_reg)
predictor.evaluate(valid_data_reg)

                  model  score_test   score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         ExtraTreesMSE -108.024002 -132.139915        0.446296       0.076599    3.255411                 0.446296                0.076599           3.255411            1       True          7
1       RandomForestMSE -109.164822 -134.085114        0.419242       0.078687    6.702206                 0.419242                0.078687           6.702206            1       True          5
2   WeightedEnsemble_L2 -109.276244 -128.219325        3.257599       1.420413  232.412809                 0.022486                0.002624           0.360921            2       True         12
3              CatBoost -110.617249 -134.897091        0.030922       0.010550  162.453468                 0.030922                0.010550         162.453468            1       True          6
4         LightGBMLarge -111.3

Evaluation: mean_absolute_error on test data: -109.27624442550133
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -109.27624442550133,
    "root_mean_squared_error": -220.83813362474177,
    "mean_squared_error": -48769.481262859365,
    "r2": 0.9505668121922949,
    "pearsonr": 0.9749931626958129,
    "median_absolute_error": -53.19999999999999
}


{'mean_absolute_error': -109.27624442550133,
 'root_mean_squared_error': -220.83813362474177,
 'mean_squared_error': -48769.481262859365,
 'r2': 0.9505668121922949,
 'pearsonr': 0.9749931626958129,
 'median_absolute_error': -53.19999999999999}

In [11]:
writePredictionsToFile(modelName = 'WeightedEnsemble_L2', filename = 'WeightedEnsemble_L2')


In [19]:
writePredictionsToFile(modelName = 'CatBoost_BAG_L1/T2', filename = 'CatBoost_BAG_L1_T2')
writePredictionsToFile(modelName = 'LightGBMXT_BAG_L1/T2', filename = 'LightGBMXT_BAG_L1_T2')
writePredictionsToFile(modelName = 'NeuralNetTorch_BAG_L2/T2', filename = 'NeuralNetTorch_BAG_L2_T2')

In [ ]:
#predictor.refit_full()